In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import calendar

In [5]:
#paths to kiki and will data 
k_disc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_disc"
k_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_chase"
k_ally_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\kiki_ally"
# k_end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\k_data\k_budget_data.xlsx"
w_pnc_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\pnc_csvs"
W_chase_path = r"C:\Users\willd\OneDrive\Desktop\Budget\statements\chase_csvs"
end_path = r"C:\Users\willd\OneDrive\Desktop\Budget\clean_data\combined_budget_data.xlsx"

In [9]:
# combine all csvs in a path return df  // with no duplicates 
def combine_csvs(path):
    df_list = []
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        df = pd.read_csv(full_path)
        df_list.append(df)
    combined = pd.concat(df_list)
    combined.columns.values[0] = "Date"
    no_dup = combined.drop_duplicates()
    return no_dup


# remove specific payments from a df
def remove_item(df, descript):
    remove_df = df[~df['Description'].str.contains(descript)]
    return remove_df
    
    
# reformat to only date amount desc, category columns
def column_remove(df, user, bank):
    df.columns = df.columns.str.replace(' ', '') 
    if 'Category' not in df.columns:
        df["Category"] = 'misc'
    df["User"] = user
    df["Bank"] = bank
    df["In/Out"] = 'tbd'
    new = df[['Date', 'Amount', 'Description', 'Category',"User","Bank","In/Out"]]
    return new

# assign transaction catagory
def category_assign(df):
    # make seriess 
    s1 = df['Description']
    s2 = df['Amount']
    #ks category conditions and values
    k_cat_condition = [(s1.str.contains('VENMO'))&(s2 < (-600)),(s1.str.contains('VENMO')),
                       (s1.str.contains('COMCAST')),
                       (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
    k_cat_value = ['RENT','VENMO','Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']
    df['Category'] = np.select(k_cat_condition, k_cat_value)
    df.loc[df['Category'] == '0', 'Category'] = 'Misc'
    return df 



In [20]:
# # ws categories conditions and values
# w_cat_condition = [(df.str.contains('VENMO')) & (df2 > 0) & (df2 < 600),(df.str.contains('VENMO'))& (df2 > 0) & (df2 > 600), (df.str.contains('VENMO')) & (df2 < 0), (df.str.contains('SPOTIFY') | df.str.contains('APPLE')),
#                  (df.str.contains('ACTALENT')), (df.str.contains('CHECK'))
#                 , (df.str.contains('SCHWAB BROK'))]
# w_cat_value = ['VENMO IN','KIKI RENT','VENMO OUT', 'SUBSCRIP', 'Belair', 'RENT', 'SCHWAB']
# s1 = df['Description']
# s2 = df['Amount']
# k_cat_condition = [(s1.str.contains('VENMO'))& (s2 < 600),(s1.str.contains('VENMO'))&(s2 < -600),
#                    (s1.str.contains('COMCAST')),
#                    (s1.str.contains('Primco')), (s1.str.contains('SCHWAB')), (s1.str.contains('Interest Paid'))]
# k_cat_value = ['VENMO','RENT', 'Bills & Utilities', 'Prim and Co', 'Investment', 'Intrest']

In [10]:
k_disc = column_remove(combine_csvs(k_disc_path), 'K', 'disc')
k_chase = column_remove(combine_csvs(k_chase_path), 'K', 'chase')
k_ally = column_remove(combine_csvs(k_ally_path), 'K', 'ally')
# categorize ally transactions
k_ally = category_assign(k_ally)
#remove cc payments from ally
ally_no_cc = remove_item(k_ally, 'CHASE CREDIT CRD')
chase_no_cc = remove_item(k_chase, 'AUTOMATIC PAYMENT')
disc_no_cc = remove_item(k_disc, 'DIRECTPAY FULL BALANCESEE DETAILS')
# make remoining discover 
disc_no_cc['Amount'] = 0 - disc_no_cc['Amount']

C:\Users\willd\AppData\Local\Temp\ipykernel_23516\4234097929.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disc_no_cc['Amount'] = 0 - disc_no_cc['Amount']


In [26]:

full_df =  pd.concat([disc_no_cc, chase_no_cc, ally_no_cc])

# change date dtype
full_df['Date'] = pd.to_datetime(full_df['Date'])
full_df['Month'] = pd.DatetimeIndex(full_df['Date']).month
full_df = full_df.sort_values(by='Date')

full_df = full_df[full_df['Date'] >= '2023-09-01']
# change month colum to date
full_df['Month'] = full_df['Month'].apply(lambda x: calendar.month_abbr[x])
full_df = full_df.sort_values(by='Date')

full_df = full_df.reset_index(drop=True)

full_df.loc[full_df['Amount'] < 0, "In/Out"] = 'out'
full_df.loc[full_df['Amount'] > 0, "In/Out"] = 'in'
full_df.loc[full_df['Amount'] < 0, 'Amount'] = 0 - full_df['Amount']

In [27]:
full_df

,Date,Amount,Description,Category,User,Bank,In/Out,Month
0,2023-09-01,31.27,IKEA CENTENNIAL,Home,K,chase,out,Sep
1,2023-09-03,1149.00,IKON PASS,Entertainment,K,chase,out,Sep
2,2023-09-03,200.63,Interest Paid,Intrest,K,ally,in,Sep
3,2023-09-05,945.42,VENMO CASHOUT,VENMO,K,ally,in,Sep
4,2023-09-06,24.98,KING SOOPERS 0068,Groceries,K,chase,out,Sep
...,...,...,...,...,...,...,...,...
132,2023-12-21,5.00,UBER TRIP,Travel,K,chase,out,Dec
133,2023-12-21,42.73,UBER TRIP,Travel,K,chase,out,Dec
134,2023-12-27,43.54,LEVIS.COM,Shopping,K,chase,out,Dec
135,2023-12-28,25.00,WALL NAILS &amp; SPA LLC,Personal,K,chase,out,Dec


In [28]:
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
full_df.to_excel(writer, sheet_name="full_df")
# monthly_ratio.to_excel(writer, sheet_name="all_mon")
# money_in.to_excel(writer, sheet_name="income")
# money_out.to_excel(writer, sheet_name="expense")
writer.close()